In [20]:
from neo4j import GraphDatabase

uri = "neo4j://10.195.2.227:7687"
username = "neo4j"
password = "neo4j-connect"

driver = GraphDatabase.driver(uri, auth=(username, password))

def add_node(session, node):
    result = session.run("CREATE (a:Wiki $node)", node=node)
        
def add_edge(session, docid1, docid2, edge):
    result = session.run("MATCH (a:Wiki {docid: $docid1}) MATCH (b:Wiki {docid: $docid2}) MERGE (a)-[:$edge]->(b)", docid1=docid1, docid2=docid2, edge=edge)

In [21]:
import json
from tqdm import tqdm
import time
import gc

s_time = time.time()
data = json.load(open("C:\\Users\\96181\\Desktop\\Master's Thesis\\T-Rex-Sample-Data.json", "r+"))
with driver.session() as session:
    for d in tqdm(data):
        entry = {
            'docid': d['docid'],
            'title': d['title'],
            'text': d['text'],
            'uri': d['uri']
        }
        add_node(session, entry)
        
print(f"Time for file {file} is {int(e_time - time.time())}sec")
del data
gc.collect()

  6%|████▍                                                                         | 567/10000 [00:43<12:00, 13.09it/s]


BufferError: Existing exports of data: object cannot be re-sized

In [50]:
data[0]['triples'][0]

{'sentence_id': 0,
 'predicate': {'boundaries': None,
  'surfaceform': None,
  'uri': 'http://www.wikidata.org/prop/direct/P31',
  'annotator': 'NoSubject-Triple-aligner'},
 'object': {'boundaries': [94, 109],
  'surfaceform': 'language family',
  'uri': 'http://www.wikidata.org/entity/Q25295',
  'annotator': 'Wikidata_Spotlight_Entity_Linker'},
 'dependency_path': None,
 'confidence': None,
 'subject': {'boundaries': [4, 27],
  'surfaceform': 'Austroasiatic languages',
  'uri': 'http://www.wikidata.org/entity/Q33199',
  'annotator': 'Wikidata_Spotlight_Entity_Linker'},
 'annotator': 'NoSubject-Triple-aligner'}

In [38]:
def add_edge(session, docid1, docid2, edge):
    edge_name = edge['predicate_name'].replace(' ', '_')
    result = session.run("MATCH (a:Wiki {docid: $docid1}) MATCH (b:Wiki {docid: $docid2}) MERGE (a)-[r:$edge_name]->(b) SET r += $edge", docid1=docid1, docid2=docid2, edge=edge, edge_name=edge_name)

predicate_names = {}
def get_predicate_name(uri):
    pred_id = uri.split('/')[-1]
    if pred_id in predicate_names:
        return predicate_names[pred_id]
    
    url = f"https://www.wikidata.org/w/api.php?action=wbgetentities&ids={prop_id}&format=json"
    response = requests.get(url)
    label = response.json()['entities'][prop_id]['labels']['en']['value']
    predicate_names[pred_id] = label
    return label

with driver.session() as session:
    t = data[0]['triples'][0]
    predicate_name = get_predicate_name(t['predicate']['uri'])
    entry = {
        'predicate_surfaceform': t['predicate']['surfaceform'],
        'predicate_uri': t['predicate']['uri'],
        'predicate_name': predicate_name
        'object_surfaceform': t['object']['surfaceform'],
        'subject_surfaceform': t['subject']['surfaceform'],  
    }
    add_edge(session, t['subject']['uri'], t['object']['uri'], entry)

{'sentence_id': 1,
 'predicate': {'boundaries': None,
  'surfaceform': None,
  'uri': 'http://www.wikidata.org/prop/direct/P17',
  'annotator': 'NoSubject-Triple-aligner'},
 'object': {'boundaries': [17, 25],
  'surfaceform': 'American',
  'uri': 'http://www.wikidata.org/entity/Q30',
  'annotator': 'Wikidata_Spotlight_Entity_Linker'},
 'dependency_path': None,
 'confidence': None,
 'subject': {'boundaries': None,
  'surfaceform': 'Apple Inc.',
  'uri': 'http://www.wikidata.org/entity/Q312',
  'annotator': 'NoSubject-Triple-aligner'},
 'annotator': 'NoSubject-Triple-aligner'}

In [24]:
data[0]['triples'][0]['predicate']

{'boundaries': None,
 'surfaceform': None,
 'uri': 'http://www.wikidata.org/prop/direct/P31',
 'annotator': 'NoSubject-Triple-aligner'}

In [43]:
data[0]['triples'][0]

{'sentence_id': 0,
 'predicate': {'boundaries': None,
  'surfaceform': None,
  'uri': 'http://www.wikidata.org/prop/direct/P31',
  'annotator': 'NoSubject-Triple-aligner'},
 'object': {'boundaries': [94, 109],
  'surfaceform': 'language family',
  'uri': 'http://www.wikidata.org/entity/Q25295',
  'annotator': 'Wikidata_Spotlight_Entity_Linker'},
 'dependency_path': None,
 'confidence': None,
 'subject': {'boundaries': [4, 27],
  'surfaceform': 'Austroasiatic languages',
  'uri': 'http://www.wikidata.org/entity/Q33199',
  'annotator': 'Wikidata_Spotlight_Entity_Linker'},
 'annotator': 'NoSubject-Triple-aligner'}

In [45]:
data[0]['triples'][0]['predicate']['uri'].split('/')[-1]

'P31'

In [28]:
data[0]['triples'][0]['subject']

{'boundaries': [4, 27],
 'surfaceform': 'Austroasiatic languages',
 'uri': 'http://www.wikidata.org/entity/Q33199',
 'annotator': 'Wikidata_Spotlight_Entity_Linker'}

In [48]:
import requests

prop_id = 'P31'
url = f"https://www.wikidata.org/w/api.php?action=wbgetentities&ids={prop_id}&format=json"
response = requests.get(url)
label = response.json()['entities'][prop_id]['labels']['en']['value']
label

'instance of'

In [49]:
label.replace(' ', '_')

'instance_of'